## Постановка задачи

Дано множество прямоугольников с ребрами, параллельными осям. Дан прямоугольник запроса с таким же свойством. Необходимо найти все прямоугольники из множества, которые пересекаются с прямоугольником запроса ребрами. Утверждается, что нет прямоугольников, чьи концы лежат внутри прямоугольника запроса.

<img src="img/1_slide.svg">

Сведем задачу к меньшей:<br>
Пусть есть множество отрезков таких, что каждый отрезок параллелен како-либо оси, и прямоугольник запроса. Необходимо ответить, какие отрезки пересекают данный прямоугольник. Известно, что нет отрезка, чей конец лежит внутри прямоугольника, следовательно, отрезок должен пересекать либо две горизонтальные, либо две вертикальные грани. Поэтому достаточно проверить перечение отрезков с одной горизотальной и с одной вертикальной гранями. Рассмотрим, например, пересечение с вертикальной (с горизонтальной задача решается аналогично). 

Итого мы имеем:
* Множество горизонтальных отрезков
* Один вертикальный отрезок

И нам необходимо найти подмножество отрезков, пересекающих данный вертикальный.

Решить данную задачу можно используя следующие структуры данных:
* Дерево интервалов (interval tree)
* Приоритетное дерево поиска (priority search tree)

## Дерево интервалов (interval tree)

Интервальное дерево, $Interval\_Tree(S)$, для $S$ — это дополненное бинарное дерево, в котором каждая вершина $v$ имеет следующее строение:
* ключ, $v.key$
* два указателя, $v.left$ и $v.right$, на левое и правое поддеревья
* вспомогательный указатель, $v.aux$, на дополнительную структуру.

### Пример

<img src="img/2_slide.svg">

### Построение дерева интервалов

**function** Interval\_Tree(S)

**Pseudocode:**
1. Если $S == \varnothing$, вернуть $nil$.
2. Создать вершину $v$ такую, что $v.key$ равен $x$, где $x$ — это средняя точка множества концов, такая что $|S|/2$ концов меньше $x$ и столько же больше. Определим $S_{l} = \{I_{i}|r_{i} < x\}$, $S_{r} = \{I_{j}|x \lt l_{j}\}$ и $S_{m} = \{I_{k}|r_{k} \lt x \lt r_{k}\}$.
3. Присвоим $v.left = Interval\_Tree(S_{l})$
4. Присвоим $v.right = Interval\_Tree(S_{r})$ 
5. Создать вершину $w$, которая является корневой вершиной вспомогательной структуры, ассоциированной с множеством $S_{m}$, такой, что $w.left$ и $w.right$ указывают на два отсортированных массива, $SA(S_{m}.left)$ и $SA(S_{m}.right)$. $SA(S_{m}.left)$ обозначает массив левых концов интервалов из $S_{m}$ в возрастающем порядке, и $SA(S_{m}.right)$ — массив правых концов интервалов из $S_{m}$ в порядке убывания.
6. Множество $v.aux$ приравнять к вершине $w$.

### Применение

**Проблема поиска прилежащего интервала (Enclosing Interval Searching Problem)** Даны множество $S$, содержащее $n$ интервалов, и точка $q$. Необходимо найти все интервалы, содержащие $q$, то есть найти подмножество $F \subseteq S$ такое, что $F = \{I_{i}|l_{i} \leqslant q \leqslant r_{i}\}$.

**Проблема поиска перекрывающего интервала (Overlapping Interval Searching Problem)** Даны множество $S$, содержащее $n$ интервалов, и интервал $Q$. Найти все интервалы из $S$, пересекающиеся с $Q$, то есть найти подмножество $F \subseteq S$ такое, что $F = \{I_{i}|I_{i} \cap Q \neq \varnothing \}$.

#### Теорема
> **Проблема поиска прилежащего интервала (Enclosing Interval Searching Problem)** и **Проблема поиска перекрывающего интервала (Overlapping Interval Searching Problem)** для множества $S$, состоящего из $n$ интервалов, могут быть решены за $O(\log n)$ (плюс время для вывода), используя $O(n)$ памяти.

## Модификация для решения данной задачи

Вместо двух отсортированных массивов будем хранить декартово дерево концов отрезков. В нем мы будем делать запросы по бесконечному с одной стороны прямоугольнику: $(-\infty, q_{x}] \times [q_{y}, q_{y}^{'}]$, если $q_{y} \lt x_{mid}$, и $[q_{x}, \infty) \times [q_{y}, q_{y}^{'}]$ иначе.

<img src="img/modification.svg">

#### Теорема
> Дерево интервалов c декартовым деревом вместо списков будет занимать $O(n \log n)$ памяти, построится за $O(n \log n)$, и будет отвечать на запрос за $O(\log^{2}n + k)$

$\triangleright$<div style="padding-left:40px">
Память: каждый $v_{aux}$ занимает $O(n_{aux} \log n_{aux})$ памяти, $\sum n_{aux} = n$, значит, $\sum |v.aux| = O(n \log n)$<br>
Время построения: дерево отрезков строится за $O(n \log n)$, как и 2 отсортированных списка, поэтому
предыдущая оценка работает.<br>
Время запроса: запрос в декартовом дереве занимает $O(\log n + k)$, всего таких деревьев $O(\log n)$, поэтому суммарное время запроса $O(\log^{2} n + k)$.
</div>$\triangleleft$

## Приоритетное дерево поиска (Priority Search Tree)

Вместо декартового дерева можно использовать структуру, которая специально заточена на обработку запросов в виде бесконечных "стаканов". Рассмотрим Приоритетное дерево поиска. Приоритетное дерево поиска изначально было введено McCreight. Это гибрид двух структур: бинарное дерево поиска и приоритетная очередь.

Приоритетное дерево поиска может быть использовано для поддержки запросов поиска, среди множества $S$, состоящего из $n$ точек, точки $p$ с минимальным $p.y$ таким, что его $x$ координата лежит в заданном промежутке $[l,r]$, то есть $l \leqslant p.x \leqslant r.$ Как будет показано позже, ответ на данный запрос может быть получен за $O(\log n).$

### Построение приоритетного дерева поиска

**function** $Priority\_Search\_Tree(S)$ <br> 

**Method:**
1. Если $S = \varnothing$, то завершить работу
2. Создать вершину $v$ такую, что $v.key$ равен среднему значению множества $\{p.x|p \in S\}$, и $v.aux$ содержит индекс $i$ точки $p_{i}$, чья $y$ координата является минимальной среди всех $y$ множества $S$, то есть $p_{i}.y = min\{p.y|p \in S\}$.
3. Пусть $S_{l} = \{p \in S \backslash \{p_{v.aux} \} | p.x \leqslant x.key \}$ и $S_{r} = \{p \in S \backslash \{p_{v.aux} \} | p.x \gt x.key \}$ обозначают множества точек, чьи $x$ координаты меньше или равны и больше, чем $v.key$.
4. $v.left = Priority\_Search\_Tree\_root(S_{l})$.
5. $v.right = Priority\_Search\_Tree\_root(S_{r})$.
6. $return \space v$.

#### Теорема
> Приоритетное дерево поиска для множества $S$, состоящего из $n$ точек из $R^2$, может быть построено за $O(n\log n)$, используя $O(n)$ памяти.

$\triangleright$<div style="padding-left:40px">
Память: каждая вершина соответствует одной точке, таким образом необходимо $O(n)$ памяти.<br>
Время построения: сортировка по иксу работает $O(n\log n)$. Шаг алгоритма без учета рекурсивных вызовов работает за $O(n)$, а глубина рекурсии $O(\log n)$. Итого $O(n\log n)$.
</div>$\triangleleft$

### Пример

<img src="img/priority_search_tree.svg">

### Применение

**Grounded 2D Range Search Problem** Дано множество $S$ из $n$ точек из плоскости $R^2$. С возможностью препроцессинга, нати подмножество $F$ из точек, чьи $x$ и $y$ координаты лежат в области $[x_{l}, x_{r}] \times [-\infty, y_{r}], x_{l}, x_{r}, y_{r} \in R, x_{l} \leqslant x_{r}$.

#### Теорема
> **Grounded 2D Range Search Problem** для множества $S$, состоящего из $n$ точек из $R^2$, может быть решена за $O(\log n)$ (плюс время для вывода), с постройкой приоритетного дерева поиска для $S$ за $O(n\log n)$ и использованием $O(n)$ памяти.

#### Теорема
> Интервальное дерево с двумя приоритетными деревьями поиска вместо списков будет занимать $O(n)$ памяти, построится за $O(n\log n)$, и будет отвечать на запрос за $O(\log^2 n + k)$.